In [35]:
import cvxpy as cp
import pandas as pd
import numpy as np
import cplex
import scipy.stats as stats

In [125]:
x_n = 2
n = 20000
gamma = 3
rho = 0.6

np.random.seed(x_n)

market_ = np.random.randn(n, 30)
brownian_ = np.random.randn(n, 30)
hc_ = rho * market_ + np.sqrt(1 - rho ** 2) * brownian_

risky_ret = cp.exp(0.1+(0.5*(0.20**2))+0.20*market_[:, 0])
rf_ret = cp.exp(0.05*np.ones(market_[:, 0].shape))

h_ret = 5000*np.cumprod(np.exp(0.04+0.03*(rho * market_ + np.sqrt(1 - rho ** 2) * brownian_)), axis=1)
dc_rate = cp.exp((0.04 + 0.04) * (np.array(range(30)) + 1) * (-1))

x = cp.Variable(2)

W_t = (1000+5000*0.3)*(cp.bmat([risky_ret, rf_ret]).T @ x)
H_t = h_ret @ dc_rate

obj = cp.sum(((W_t+H_t)**(1-gamma)) / (1-gamma))*(-1)
# obj = (1-gamma)*cp.log(W_t+H_t) - cp.log(1-gamma)
cons = [cp.sum(x) == 1, x >= 0]#, W_t == (1000+5000*0.3)*(cp.bmat([risky_ret, rf_ret]).T @ x)]

prob = cp.Problem(cp.Minimize(obj), cons)
prob.solve(cp.SCS, verbose=True)
print(np.round(x.value, 3)*100)

----------------------------------------------------------------------------
	SCS v2.1.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 180004
eps = 1.00e-04, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 0, rho_x = 1.00e-03
Variables n = 40002, constraints m = 120003
Cones:	primal zero / dual free vars: 1
	linear vars: 2
	soc vars: 120000, soc blks: 40000
WARN: aa_init returned NULL, no acceleration applied.
Setup time: 6.74e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.42e+00  3.25e+03  7.26e-01 -2.93e+07 -4.65e+06  8.13e-09  1.59e-02 
   100| 4.69e-03  1.32e+02  8.79e-01  6.10e+04  3.91e+03  1.09e-09  4.19e-01 

In [45]:
from scipy.optimize import minimize

In [ ]:
x_n = 2
n = 20000
gamma = 10
rho = 0.6

np.random.seed(x_n)

market_ = np.random.randn(n, 30)
brownian_ = np.random.randn(n, 30)
hc_ = rho * market_ + np.sqrt(1 - rho ** 2) * brownian_

risky_ret = cp.exp(0.1+(0.5*(0.20**2))+0.20*market_[:, 0])
rf_ret = cp.exp(0.05*np.ones(market_[:, 0].shape))

h_ret = 5000*np.cumprod(np.exp(0.04+0.03*(rho * market_ + np.sqrt(1 - rho ** 2) * brownian_)), axis=1)
dc_rate = cp.exp((0.04 + 0.04) * (np.array(range(30)) + 1) * (-1))

x = cp.Variable(2)

W_t = (1000+5000*0.3)*(cp.bmat([risky_ret, rf_ret]).T @ x)
H_t = h_ret @ dc_rate

obj = cp.sum((W_t+H_t))# **(1-gamma) / (1-gamma)) / n
# obj = (1-gamma)*cp.log(W_t+H_t) - cp.log(1-gamma)
cons = [cp.sum(x) == 1, x >= 0]#, W_t == (1000+5000*0.3)*(cp.bmat([risky_ret, rf_ret]).T @ x)]

prob = cp.Problem(cp.Maximize(obj), cons)
prob.solve(cp.CPLEX, verbose=True)
print(np.round(x.value, 3)*100)

In [115]:
gamma = 10
period = 5
initial_wealth = 100000
initial_income = 10000
x_n = 1

In [117]:
def obj(x, x_n, n, rho, gamma):
    np.random.seed(x_n)

    market_ = stats.norm.rvs(size=(n, period), loc=0, scale=1)
    brownian_ = stats.norm.rvs(size=(n, period), loc=0, scale=1)
    hc_ = rho * market_ + np.sqrt(1 - rho ** 2) * brownian_

    risky_ret = np.exp(0.1+(0.5*(0.20**2))+0.20*market_[:, 0])
    rf_ret = np.exp(0.05*np.ones(market_[:, 0].shape))

    h_ret = initial_income*np.cumprod(np.exp(0.04+0.03*(rho * market_ + np.sqrt(1 - rho ** 2) * brownian_)), axis=1)
    dc_rate = np.exp((0.04 + 0.04) * (np.array(range(period)) + 1) * (-1))

    W_t = (initial_wealth+(initial_income*0.3))*(np.array([risky_ret, rf_ret]).T @ x)
    H_t = h_ret @ dc_rate
    
    obj = (((W_t+H_t) ** (1-gamma)) / (1-gamma)) * (-1)
    
    return np.mean(obj)

x = np.zeros([2])
cons = ({'type': 'ineq', 'fun': lambda x: 1 - np.sum(x)},
        {'type': 'ineq', 'fun': lambda x: x},)
optimized = minimize(obj, x, (x_n, n, rho, gamma), method='COBYLA', 
                     constraints=cons)
if not optimized.success: raise BaseException(optimized.message)
optimized.x

array([0.28419502, 0.71580498])

In [ ]:
def max_utility(sample_size, period, market_ret, market_sigma, ):
    def obj(x, n, rho, gamma, period):
        np.random.seed(1)

        market_ = stats.norm.rvs(size=(sample_size, period), loc=0, scale=1)
        brownian_ = stats.norm.rvs(size=(sample_size, period), loc=0, scale=1)
        hc_ = rho * market_ + np.sqrt(1 - rho ** 2) * brownian_

        risky_ret = np.exp():+(0.5*(market_sigma**2))+market_sigma*market_[:, 0])
        rf_ret = np.exp(0.05*np.ones(market_[:, 0].shape))

        h_ret = initial_income*np.cumprod(np.exp(0.04+0.03*(rho * market_ + np.sqrt(1 - rho ** 2) * brownian_)), axis=1)
        dc_rate = np.exp((0.04 + 0.04) * (np.array(range(period)) + 1) * (-1))

        W_t = (initial_wealth+(initial_income*0.3))*(np.array([risky_ret, rf_ret]).T @ x)
        H_t = h_ret @ dc_rate

        obj = (((W_t+H_t) ** (1-gamma)) / (1-gamma)) * (-1)

        return np.mean(obj)

    x = np.zeros([2])
    cons = ({'type': 'ineq', 'fun': lambda x: 1 - np.sum(x)},
            {'type': 'ineq', 'fun': lambda x: x},)
    optimized = minimize(obj, x, (x_n, n, rho, gamma, period), method='COBYLA', 
                         constraints=cons)
    if not optimized.success: raise BaseException(optimized.message)
    return optimized.x